In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
from IPython.display import display

In [2]:
# Define the custom R^2 metric for the Keras model
def r2_keras(output_index):
    def r2(y_true, y_pred):
        SS_res = tf.keras.backend.sum(tf.keras.backend.square(y_true[:, output_index] - y_pred[:, output_index]))
        SS_tot = tf.keras.backend.sum(tf.keras.backend.square(y_true[:, output_index] - tf.keras.backend.mean(y_true[:, output_index])))
        r2 = 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())
        return r2
    r2.__name__ = f"r2_{output_index}"
    return r2

In [3]:
# Load the input
json_input = ['48', '28', '20', '250', '18', '30', '85', '20', '17', '2.2', '2.2', '2.2', '2.2', '10', 100.0, 155.0, 184.1, 3.49, 1.39, 15.55, 3.46, 15.48, 5.2, 4.03]
# Define the inputs and output for the model
feature_names = ['Net Volume', 'Pulp Area', 'Froth surface area', 'Froth thickness',
       'Air Flow rate', 'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp',
       'k_max Ccp', 'k_max Gn', 'k_maxPo', 'k_max Sp',
       'Entrainement Savassi parameters', 'Total Solids Flow_Feed',
       'Total Liquid Flow_Feed',
       'Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed',
       'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed',
       'Pb_Feed', 'Zn_Feed']

input_df = pd.DataFrame([json_input], columns=feature_names)
display(input_df)
input_df.to_csv("inputOfModel.csv")

,Net Volume,Pulp Area,Froth surface area,Froth thickness,Air Flow rate,R_inf Ccp,R_inf Gn,R_inf Po,R_inf Sp,k_max Ccp,...,Total Solids Flow_Feed,Total Liquid Flow_Feed,Pulp Volumetric Flow_Feed,Solids SG_Feed,Pulp SG_Feed,Solids Fraction_Feed,Cu_Feed,Fe_Feed,Pb_Feed,Zn_Feed
0,48,28,20,250,18,30,85,20,17,2.2,...,100.0,155.0,184.1,3.49,1.39,15.55,3.46,15.48,5.2,4.03


In [4]:
# Load scalers
scalerX = joblib.load('scalerX_24.joblib')
scalerY = joblib.load('scalerY_23.joblib')

# Apply the scaler to the input data
scaled_input = scalerX.transform(input_df)

# Load the saved model
custom_objects = {f'r2_{i}': r2_keras(i) for i in range(23)}
model = load_model('model7_23.h5', custom_objects=custom_objects)

C:\Users\czeroo\Desktop\Managem App\SimulationApp\internship\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:

# Make predictions on the preprocessed input
y_pred = model.predict(scaled_input)

# Invert the scaling to get the original values of the predictions
y_pred = scalerY.inverse_transform(y_pred)

# Separate predictions into concentrate and tailing
concentrate = y_pred[:, [3, 4, 5, 6, 7, 8, 19, 20, 21, 22]]
tailing = y_pred[:, 9:19]

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 331ms/step


In [6]:
# Define the keys for the outputs
concentrate_keys = [
    'totalSolidFlow', 'totalLiquidFlow', 'pulpVolumetricFlow', 
    'solidsSG', 'pulpSG', 'solidsFraction', 
    'cuPercentage', 'fePercentage', 'znPercentage', 'pbPercentage'
]

tailing_keys = [
    'totalSolidFlow', 'totalLiquidFlow', 'pulpVolumetricFlow', 
    'solidsSG', 'pulpSG', 'solidsFraction', 
    'cuPercentage', 'fePercentage', 'znPercentage', 'pbPercentage'
]

# Create dataframes for concentrate and tailing
concentrate_df = pd.DataFrame(concentrate, columns=concentrate_keys)
tailing_df = pd.DataFrame(tailing, columns=tailing_keys)

# Display the dataframes
display(concentrate_df)
display(tailing_df)

,totalSolidFlow,totalLiquidFlow,pulpVolumetricFlow,solidsSG,pulpSG,solidsFraction,cuPercentage,fePercentage,znPercentage,pbPercentage
0,193.55983,184.913101,314.806366,5.83245,1.406989,11.47348,7.819666,23.483473,53.844097,4.851144


,totalSolidFlow,totalLiquidFlow,pulpVolumetricFlow,solidsSG,pulpSG,solidsFraction,cuPercentage,fePercentage,znPercentage,pbPercentage
0,25.719416,106.410049,94.665367,4.038092,2.189166,42.752033,7.104559,26.736698,9.65136,8.587224


In [7]:
# Define the output columns for the prediction table
output_columns = [
    'Air Efficiency', 'Flotation Rate: Cell 1', 'Entrainment: Cell 1', 
    'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 
    'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 
    'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 
    'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 
    'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 
    'Pulp SG_Tailings', 'Solids Fraction_Tailings', 
    'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails', 
    'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate'
]

# Create and display the prediction table
prediction_table = pd.DataFrame(y_pred, columns=output_columns)
display(prediction_table)

# Save the prediction table to a CSV file
prediction_table.to_csv('OutputOfSecondCell.csv', index=False)

,Air Efficiency,Flotation Rate: Cell 1,Entrainment: Cell 1,Total Solids Flow_Concentrate,Total Liquid Flow_Concentrate,Pulp Volumetric Flow_Concentrate,Solids SG_Concentrate,Pulp SG_Concentrate,Solids Fraction_Concentrate,Total Solids Flow_Tailings,...,Pulp SG_Tailings,Solids Fraction_Tailings,Cu_Tails,Fe_Tails,Pb_Tails,Zn_Tails,Cu_Concentrate,Fe_Concentrate,Pb_Concentrate,Zn_Concentrate
0,213.212067,0.887549,0.377299,193.55983,184.913101,314.806366,5.83245,1.406989,11.47348,25.719416,...,2.189166,42.752033,7.104559,26.736698,9.65136,8.587224,7.819666,23.483473,53.844097,4.851144
